# Convert the TensorFlow-Trained BERT Model to a PyTorch Model
We do this to show HuggingFace's ability to convert models between TensorFlow and PyTorch.

We will subsequently deploy the model as a PyTorch model using the TorchServe Inference Container.

In [1]:
import boto3
import sagemaker
import pandas as pd

sess = sagemaker.Session()
bucket = sess.default_bucket()
role = sagemaker.get_execution_role()
region = boto3.Session().region_name

sm = boto3.Session().client(service_name="sagemaker", region_name=region)

In [2]:
%store -r training_job_name

In [3]:
try:
    training_job_name
except NameError:
    print("+++++++++++++++++++++++++++++++")
    print("[ERROR] Please wait for the Training notebook to finish.")
    print("+++++++++++++++++++++++++++++++")

In [4]:
print("Previous training_job_name: {}".format(training_job_name))

Previous training_job_name: tensorflow-training-2024-03-03-04-02-13-539


# Download the TensorFlow-Trained Model from S3 to this Notebook

In [5]:
models_dir = "./model"

In [6]:
!aws s3 cp s3://$bucket/$training_job_name/output/model.tar.gz $models_dir/model.tar.gz

download: s3://sagemaker-us-east-1-211125778552/tensorflow-training-2024-03-03-04-02-13-539/output/model.tar.gz to model/model.tar.gz


In [7]:
import tarfile
import pickle as pkl

try:
    tar = tarfile.open("{}/model.tar.gz".format(models_dir))
    tar.extractall(path=models_dir)
    tar.close()
except Exception as e:
    print("[ERROR] in tar operation: {}".format(e))

In [8]:
!ls -al $models_dir

total 888128
drwxr-xr-x  8 sagemaker-user users       127 Mar  6 15:38 .
drwxr-xr-x 21 sagemaker-user users      4096 Mar  3 05:12 ..
drwxr-xr-x  2 sagemaker-user users        26 Mar  3 05:09 code
drwxr-xr-x  2 sagemaker-user users        57 Mar  3 05:11 metrics
-rw-r--r--  1 sagemaker-user users 909435719 Mar  3 05:11 model.tar.gz
drwxr-xr-x  4 sagemaker-user users        37 Mar  3 04:26 tensorboard
drwxr-xr-x  3 sagemaker-user users        25 Mar  3 04:04 tensorflow
drwxr-xr-x  2 sagemaker-user users        61 Mar  3 05:09 test_data
drwxr-xr-x  3 sagemaker-user users        24 Mar  3 04:04 transformers


In [9]:
transformer_model_dir = "{}/transformers/fine-tuned/".format(models_dir)

!ls -al $transformer_model_dir

total 261688
drwxr-xr-x 2 sagemaker-user users        44 Mar  3 05:06 .
drwxr-xr-x 3 sagemaker-user users        24 Mar  3 04:04 ..
-rw-r--r-- 1 sagemaker-user users       689 Mar  3 05:06 config.json
-rw-r--r-- 1 sagemaker-user users 267961728 Mar  3 05:06 tf_model.h5


In [10]:
cat $transformer_model_dir/config.json

{
  "_name_or_path": "distilbert-base-uncased",
  "activation": "gelu",
  "architectures": [
    "DistilBertForMaskedLM"
  ],
  "attention_dropout": 0.1,
  "dim": 768,
  "dropout": 0.1,
  "hidden_dim": 3072,
  "id2label": {
    "0": 1,
    "1": 2,
    "2": 3,
    "3": 4,
    "4": 5
  },
  "initializer_range": 0.02,
  "label2id": {
    "1": 0,
    "2": 1,
    "3": 2,
    "4": 3,
    "5": 4
  },
  "max_position_embeddings": 512,
  "model_type": "distilbert",
  "n_heads": 12,
  "n_layers": 6,
  "pad_token_id": 0,
  "qa_dropout": 0.1,
  "seq_classif_dropout": 0.2,
  "sinusoidal_pos_embds": false,
  "tie_weights_": true,
  "transformers_version": "4.10.0.dev0",
  "vocab_size": 30522
}


# Convert the TensorFlow Model to PyTorch

In [11]:
from transformers import DistilBertForSequenceClassification  # PyTorch version

try:
    loaded_pytorch_model = DistilBertForSequenceClassification.from_pretrained(
        transformer_model_dir,
        id2label={0: 1, 1: 2, 2: 3, 3: 4, 4: 5},
        label2id={1: 0, 2: 1, 3: 2, 4: 3, 5: 4},
        from_tf=True,
    )
except Exception as e:
    print("[ERROR] in loading model {}: ".format(e))

2024-03-06 15:38:29.181713: W tensorflow/stream_executor/platform/default/dso_loader.cc:59] Could not load dynamic library 'libcudart.so.10.1'; dlerror: libcudart.so.10.1: cannot open shared object file: No such file or directory
2024-03-06 15:38:29.182223: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2024-03-06 15:38:34.809963: W tensorflow/stream_executor/platform/default/dso_loader.cc:59] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2024-03-06 15:38:34.809995: W tensorflow/stream_executor/cuda/cuda_driver.cc:312] failed call to cuInit: UNKNOWN ERROR (303)
2024-03-06 15:38:34.810022: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (default): /proc/driver/nvidia/version does not exist
2024-03-06 15:38:34.811758: I tensorflow/core/platform/cpu_feature_gu

In [12]:
print(type(loaded_pytorch_model))
print(loaded_pytorch_model)

<class 'transformers.modeling_distilbert.DistilBertForSequenceClassification'>
DistilBertForSequenceClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0): TransformerBlock(
          (attention): MultiHeadSelfAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
   

# Save The Transformer/PyTorch Model with `.save_pretrained()`
This will create `pytorch_model.bin`

In [13]:
pytorch_models_dir = "./model/transformers/pytorch"

In [14]:
!mkdir -p $pytorch_models_dir

In [15]:
loaded_pytorch_model.save_pretrained(pytorch_models_dir)

In [16]:
!ls -al $pytorch_models_dir

total 261596
drwxr-xr-x 2 sagemaker-user users        50 Mar  6 15:38 .
drwxr-xr-x 4 sagemaker-user users        39 Mar  6 15:38 ..
-rw-r--r-- 1 sagemaker-user users       712 Mar  6 15:38 config.json
-rw-r--r-- 1 sagemaker-user users 267869890 Mar  6 15:38 pytorch_model.bin


# Load and Predict

In [17]:
from transformers import DistilBertTokenizer

tokenizer = DistilBertTokenizer.from_pretrained("distilbert-base-uncased")

In [18]:
pytorch_model = DistilBertForSequenceClassification.from_pretrained(pytorch_models_dir)

In [19]:
!ls -al $pytorch_models_dir

total 261596
drwxr-xr-x 2 sagemaker-user users        50 Mar  6 15:38 .
drwxr-xr-x 4 sagemaker-user users        39 Mar  6 15:38 ..
-rw-r--r-- 1 sagemaker-user users       712 Mar  6 15:38 config.json
-rw-r--r-- 1 sagemaker-user users 267869890 Mar  6 15:38 pytorch_model.bin


In [20]:
import torch
from transformers import DistilBertForSequenceClassification
from transformers import DistilBertConfig

config = DistilBertConfig.from_json_file("{}/config.json".format(pytorch_models_dir))

model_path = "{}/{}".format(pytorch_models_dir, "pytorch_model.bin")
model = DistilBertForSequenceClassification.from_pretrained(model_path, config=config)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

DistilBertForSequenceClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0): TransformerBlock(
          (attention): MultiHeadSelfAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)
       

In [21]:
import json
import smdebug.pytorch as smd
from torch import nn

max_seq_length = 64
classes = [1, 2, 3, 4, 5]

model.eval()

input_data = '[{"features": ["This is great!"]}, \
               {"features": ["This is bad."]}]'
print("input_data: {}".format(input_data))

data_json = json.loads(input_data)
print("data_json: {}".format(data_json))

predicted_classes = []
save_config = smd.SaveConfig(save_interval=1)
# hook = smd.Hook("/tmp/tensors", save_config=save_config, include_regex='.*')

# hook.register_module(model)

for data_json_line in data_json:
    print("data_json_line: {}".format(data_json_line))
    print("type(data_json_line): {}".format(type(data_json_line)))

    review_body = data_json_line["features"][0]
    print("""review_body: {}""".format(review_body))

    encode_plus_token = tokenizer.encode_plus(
        review_body,
        max_length=max_seq_length,
        add_special_tokens=False,
        #        return_token_type_ids=False,
        return_token_type_ids=None,
        #        pad_to_max_length=True,
        padding=True,
        return_attention_mask=True,
        return_tensors="pt",
        truncation=True,
    )

    input_ids = encode_plus_token["input_ids"]

    #    hook._write_raw_tensor_simple("input_tokens", tokenizer.tokenize(review_body))
    attention_mask = encode_plus_token["attention_mask"]

    output = model(input_ids, attention_mask)
    print("output: {}".format(output))

    softmax_fn = nn.Softmax(dim=1)
    softmax_output = softmax_fn(output[0])
    print("softmax_output: {}".format(softmax_output))

    _, prediction = torch.max(softmax_output, dim=1)

    predicted_class_idx = prediction.item()
    predicted_class = classes[predicted_class_idx]
    print("predicted_class: {}".format(predicted_class))

[2024-03-06 15:38:46.454 default:453 INFO utils.py:27] RULE_JOB_STOP_SIGNAL_FILENAME: None
[2024-03-06 15:38:46.630 default:453 INFO profiler_config_parser.py:102] Unable to find config at /opt/ml/input/config/profilerconfig.json. Profiler is disabled.
input_data: [{"features": ["This is great!"]},                {"features": ["This is bad."]}]
data_json: [{'features': ['This is great!']}, {'features': ['This is bad.']}]
data_json_line: {'features': ['This is great!']}
type(data_json_line): <class 'dict'>
review_body: This is great!
output: (tensor([[ 0.0913, -0.0083, -0.1442, -0.0366,  0.0816]],
       grad_fn=<AddmmBackward>),)
softmax_output: tensor([[0.2190, 0.1983, 0.1731, 0.1927, 0.2169]], grad_fn=<SoftmaxBackward>)
predicted_class: 1
data_json_line: {'features': ['This is bad.']}
type(data_json_line): <class 'dict'>
review_body: This is bad.
output: (tensor([[ 0.0363,  0.0183, -0.0332, -0.0609,  0.0948]],
       grad_fn=<AddmmBackward>),)
softmax_output: tensor([[0.2048, 0.2012,

# Upload Transformer/PyTorch Model to S3

In [22]:
transformer_pytorch_model_dir_s3_uri = "s3://{}/model/{}/transformer-pytorch/".format(bucket, training_job_name)
print(transformer_pytorch_model_dir_s3_uri)

s3://sagemaker-us-east-1-211125778552/model/tensorflow-training-2024-03-03-04-02-13-539/transformer-pytorch/


In [23]:
!mv ./model/transformers/pytorch/pytorch_model.bin ./model/transformers/pytorch/model.pth

In [24]:
!cd ./model/transformers/pytorch/ && tar -cvzf model.tar.gz *

config.json
model.pth


In [25]:
!aws s3 cp ./model/transformers/pytorch/model.tar.gz $transformer_pytorch_model_dir_s3_uri

upload: model/transformers/pytorch/model.tar.gz to s3://sagemaker-us-east-1-211125778552/model/tensorflow-training-2024-03-03-04-02-13-539/transformer-pytorch/model.tar.gz


In [26]:
!aws s3 ls $transformer_pytorch_model_dir_s3_uri

2024-03-06 15:39:05  246786601 model.tar.gz


In [27]:
%store transformer_pytorch_model_dir_s3_uri

Stored 'transformer_pytorch_model_dir_s3_uri' (str)


In [28]:
%store

Stored variables and their in-db values:
autopilot_endpoint_arn                                -> 'arn:aws:sagemaker:us-east-1:211125778552:endpoint
autopilot_model_arn                                   -> 'arn:aws:sagemaker:us-east-1:211125778552:model/au
autopilot_train_s3_uri                                -> 's3://sagemaker-us-east-1-211125778552/data/amazon
balance_dataset                                       -> True
balanced_bias_data_jsonlines_s3_uri                   -> 's3://sagemaker-us-east-1-211125778552/bias-detect
balanced_bias_data_s3_uri                             -> 's3://sagemaker-us-east-1-211125778552/bias-detect
bias_data_s3_uri                                      -> 's3://sagemaker-us-east-1-211125778552/bias-detect
comprehend_endpoint_arn                               -> 'arn:aws:comprehend:us-east-1:211125778552:documen
comprehend_train_s3_uri                               -> 's3://sagemaker-us-east-1-211125778552/data/amazon
comprehend_training_job_arn      

# Release Resources

In [29]:
%%html

<p><b>Shutting down your kernel for this notebook to release resources.</b></p>
<button class="sm-command-button" data-commandlinker-command="kernelmenu:shutdown" style="display:none;">Shutdown Kernel</button>
        
<script>
try {
    els = document.getElementsByClassName("sm-command-button");
    els[0].click();
}
catch(err) {
    // NoOp
}    
</script>

In [30]:
%%javascript

try {
    Jupyter.notebook.save_checkpoint();
    Jupyter.notebook.session.delete();
}
catch(err) {
    // NoOp
}

<IPython.core.display.Javascript object>